## EKS and Kubeflow 설정

### SageMaker Notebook Instance Role의 권한

![SageMakerRolePermissions](img/sagemaker_role_permissions.png)

In [1]:
import sys
print('Python Version %s' % sys.version)

Python Version 3.6.10 |Anaconda, Inc.| (default, Mar 25 2020, 23:51:54) 
[GCC 7.3.0]


In [2]:
# !pip list

In [3]:
# %%bash
# conda list

In [4]:
# %%html

# <iframe src="https://datascienceonaws.com" width="800px" height="600px"/>

# 1. Install the CLI tools

_Note:  Make sure the SageMaker role has access to create EKS Cluster._

In [5]:
%%bash

source ~/.bash_profile

#### Install `eksctl`
# To get started we'll first install the `awscli` and `eksctl` CLI tools. [eksctl](https://eksctl.io) simplifies the process of creating EKS clusters.

pip install awscli --upgrade --user

curl --silent --location "https://github.com/weaveworks/eksctl/releases/latest/download/eksctl_$(uname -s)_amd64.tar.gz" | tar xz -C /tmp

sudo mv /tmp/eksctl /usr/local/bin

eksctl version

#### Install `kubectl`
# `kubectl` is a command line interface for running commands against Kubernetes clusters. 
# Run the following to install Kubectl

curl --location -o ./kubectl https://amazon-eks.s3.us-west-2.amazonaws.com/1.15.10/2020-02-22/bin/linux/amd64/kubectl

chmod +x ./kubectl

sudo mv ./kubectl /usr/local/bin

kubectl version --short --client

#### Install `aws-iam-authenticator`

curl -o aws-iam-authenticator https://amazon-eks.s3.us-west-2.amazonaws.com/1.15.10/2020-02-22/bin/linux/amd64/aws-iam-authenticator

chmod +x ./aws-iam-authenticator

sudo mv aws-iam-authenticator /usr/local/bin

aws-iam-authenticator version

#### Install jq and envsubst (from GNU gettext utilities) 
sudo yum -y install jq gettext

#### Verify the binaries are in the path and executable
for command in kubectl jq envsubst
  do
    which $command &>/dev/null && echo "$command in path" || echo "$command NOT FOUND"
  done

echo "Completed"

0.25.0
Client Version: v1.15.10-eks-bac369
{"Version":"v0.5.0","Commit":"1cfe2a90f68381eacd7b6dcfa2bf689e76eb8b4b"}
Loaded plugins: dkms-build-requires, priorities, update-motd, upgrade-helper,
              : versionlock
Package jq-1.5-1.2.amzn1.x86_64 already installed and latest version
Package gettext-0.18.1.1-9.1.11.amzn1.x86_64 already installed and latest version
Nothing to do
kubectl in path
jq in path
envsubst in path
Completed


You should consider upgrading via the '/home/ec2-user/anaconda3/bin/python -m pip install --upgrade pip' command.
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 55.1M  100 55.1M    0     0  9748k      0  0:00:05  0:00:05 --:--:-- 11.3M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 33.6M  100 33.6M    0     0  8749k      0  0:00:03  0:00:03 --:--:-- 8747k


# 2. Configure the Environment Variables

In [6]:
%%bash

source ~/.bash_profile

export AWS_REGION=$(aws configure get region)
echo "export AWS_REGION=${AWS_REGION}" | tee -a ~/.bash_profile

export AWS_CLUSTER_NAME=cluster
echo "export AWS_CLUSTER_NAME=${AWS_CLUSTER_NAME}" | tee -a ~/.bash_profile

echo "Completed"

export AWS_REGION=ap-northeast-2
export AWS_CLUSTER_NAME=cluster
Completed


# 3. Create the EKS Cluster

_Note:  Creating a cluster may take about 15 minutes.  Please be patient._

In [7]:
%%bash

source ~/.bash_profile

cat << EOF > cluster.yaml
---
apiVersion: eksctl.io/v1alpha5
kind: ClusterConfig

metadata:
  name: ${AWS_CLUSTER_NAME}
  region: ${AWS_REGION}

cloudWatch:
  clusterLogging:
    enableTypes: ["*"]

managedNodeGroups:
- name: cpu-nodes
  instanceType: c5.xlarge
  volumeSize: 100
  desiredCapacity: 2
  iam:
    withAddonPolicies:
      albIngress: true

#secretsEncryption:
#  keyARN: ${MASTER_ARN}
EOF

In [8]:
!cat cluster.yaml

---
apiVersion: eksctl.io/v1alpha5
kind: ClusterConfig

metadata:
  name: cluster
  region: ap-northeast-2

cloudWatch:
  clusterLogging:
    enableTypes: ["*"]

managedNodeGroups:
- name: cpu-nodes
  instanceType: c5.xlarge
  volumeSize: 100
  desiredCapacity: 2
  iam:
    withAddonPolicies:
      albIngress: true

#secretsEncryption:
#  keyARN: 


In [10]:
%%bash

source ~/.bash_profile

eksctl create cluster -f ./cluster.yaml

echo "Completed"

[ℹ]  eksctl version 0.25.0
[ℹ]  using region ap-northeast-2
[!]  retryable error (EC2MetadataError: failed to make EC2Metadata request
	status code: 405, request id: 
caused by: ) from ec2metadata/GetToken - will retry after delay of 44.725214ms
[!]  retryable error (EC2MetadataError: failed to make EC2Metadata request
	status code: 405, request id: 
caused by: ) from ec2metadata/GetToken - will retry after delay of 105.564368ms
[!]  retryable error (EC2MetadataError: failed to make EC2Metadata request
	status code: 405, request id: 
caused by: ) from ec2metadata/GetToken - will retry after delay of 194.404196ms
[!]  retryable error (EC2MetadataError: failed to make EC2Metadata request
	status code: 405, request id: 
caused by: ) from ec2metadata/GetToken - will retry after delay of 307.34848ms
[!]  retryable error (EC2MetadataError: failed to make EC2Metadata request
	status code: 405, request id: 
caused by: ) from ec2metadata/GetToken - will retry after delay of 942.646816ms
[!]  re

# _Please Wait 15-20 Minutes for the EKS Cluster to Start Successfully_

# _Your output will look like this..._

![](img/verify_eks.png)

# You will see retry errors. This is OK.